# Importing the libraries

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Defining the GAN

In [2]:
def define_GAN(g_model, d_model):
    d_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

## Defining the Discriminator

In [5]:
def define_discriminator(input_shape= (160,160,3)):
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))

    # 160 * 160 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))

    # 80 * 80 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 40 * 40 * 3
    model.add(keras.layers.Conv2D(filters= 256,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 20 * 20 * 3
    model.add(keras.layers.Conv2D(filters= 512,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 10 * 10 * 3

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(units= 1,
                                 activation= 'sigmoid'))
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])
    
    return model

## Defining the Generator

In [9]:
def define_generator(latent_dim):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(units= 1024 * 10 * 10, input_dim= latent_dim))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Reshape((10, 10, 1024)))
    # 10 * 10
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 20 * 20
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 40 * 40
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 80 * 80
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 160 * 160
    model.add(keras.layers.Conv2DTranspose(filters= 256,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    
    # Ready for colorization
    model.add(keras.layers.Conv2D(filters= 3,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  activation= 'tanh'))
    
    return model

## Generating the Fake samples

In [14]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape((n_samples, latent_dim))
    return X

In [15]:
def generate_fake_sample(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim= latent_dim,
                                     n_samples= n_samples)
    
    X = g_model.predict(x_input)
    y = np.ones((n_samples, 1))
    return X, y

## Generating the Real samples

In [26]:
def generate_real_sample(train_it):
    X, _ = train_it.next()
    X = X.astype('float32')
    #X = (X - 127.5) / 127.5
    y = np.ones((X.shape[0], 1))
    return X, y

## Saving the Plot and summarizing

In [11]:
def save_plot(x_input, epoch, n=7):
    x_input = (x_input + 1.0) / 2.0
    filename = f'generated_{epoch + 1}.png'
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.imshow(x_input[i,:,:,:])
        plt.axis('off')
    plt.savefig(filename)
    plt.close()

In [12]:
def summarize_the_model(g_model, d_model, epoch, latent_dim, n_samples):
    X_real, y_real = generate_real_sample(n_samples= n_samples)
    X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                          latent_dim= latent_dim,
                                          n_samples= n_samples)
    print(f'Accuracy on real data: {d_model.evaluate(X_real, y_real, verbose= 0)}')
    print(f'Accuracy on fake data: {d_model.evaluate(X_fake, y_fake, verbose= 0)}')
    filename = f'model_e_{epoch + 1}.h5'
    save_plot(x_input= X_fake,
              epoch= epoch)
    
    g_model.save(filename)

# Function for training the GAN

In [13]:
def train_GAN(gan_model, g_model, d_model, dataset_len, latent_dim, iters= 100, batch_size= 256):
    half_batch = int(batch_size / 2)
    batch_per_epoch = int(dataset_len / batch_size)
    for i in range(iters):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_sample(n_samples= half_batch)
            X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                                  latent_dim= latent_dim,
                                                  n_samples= half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            dloss = d_model.train_on_batch(X, y)
            
            x_gan = generate_latent_points(latent_dim= latent_dim,
                                             n_samples= batch_size)
            y_gan = np.ones((batch_size, 1))
            gloss = gan_model.train_on_batch(x_gan, y_gan)
            print(f'Epoch: {i+1}, Batches/Epoch: {j+1}/{batch_per_epoch}, gloss: {gloss}, dloss: {dloss}')
        # After each epoch
        summarize_the_model(g_model= g_model,
                            d_model= d_model,
                            epoch= i,
                            latent_dim= latent_dim,
                            n_samples= batch_size)

## Training the Model

## Loading the data

In [17]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
import zipfile, os
with zipfile.ZipFile(os.getcwd() + '/celeba.zip', 'r') as zip_ref:
  zip_ref.extractall(os.getcwd()+'/data_faces')

--2020-06-21 08:42:07--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.184
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  21.4MB/s    in 66s     

2020-06-21 08:43:14 (20.8 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]



In [24]:
datagen = keras.preprocessing.image.ImageDataGenerator()
path = os.getcwd() + '/data_faces'
train_it = datagen.flow_from_directory(path,
                                       batch_size= 4,
                                       target_size= (218, 178))

Found 202599 images belonging to 1 classes.


In [29]:
X, y = generate_real_sample(train_it= train_it)
X.shape, y.shape

((4, 218, 178, 3), (4, 1))